In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,39333
2,Huelva,Confirmados PDIA 14 días,3188
3,Huelva,Tasa PDIA 14 días,"621,2366272385369"
4,Huelva,Confirmados PDIA 7 días,1596
5,Huelva,Tasa PDIA 7 dias,"311,0080480152776"
6,Huelva,Total Confirmados,39540
7,Huelva,Curados,34283
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  39333.0


/tmp/ipykernel_3535/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11020.0


/tmp/ipykernel_3535/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3535/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3535/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3535/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 297 personas en los últimos 7 días 

Un positivo PCR cada 150 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,39333.0,1596.0,3188.0,513170.0,311.008048,621.236627,223.0
Huelva-Costa,23132.0,966.0,1969.0,289548.0,333.623441,680.025419,119.0
Condado-Campiña,12312.0,450.0,827.0,156231.0,288.035025,529.344368,75.0
Huelva (capital),11020.0,485.0,962.0,143837.0,337.187233,668.812614,72.0
Sierra de Huelva-Andévalo Central,3513.0,178.0,389.0,67391.0,264.130225,577.228413,29.0
Aljaraque,1462.0,66.0,151.0,21474.0,307.348421,703.175934,12.0
Lepe,2572.0,103.0,198.0,27880.0,369.440459,710.186514,12.0
Trigueros,426.0,45.0,68.0,7862.0,572.373442,864.919868,11.0
Palos de la Frontera,898.0,31.0,85.0,11742.0,264.009538,723.897121,10.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,159.0,11.0,41.0,1310.0,839.694656,3129.770992,2.0
Cumbres Mayores,107.0,32.0,50.0,1749.0,1829.616924,2858.776444,8.0
Cumbres de Enmedio,1.0,1.0,1.0,51.0,1960.784314,1960.784314,0.0
Corteconcepción,34.0,8.0,10.0,536.0,1492.537313,1865.671642,0.0
Cañaveral de León,11.0,4.0,6.0,396.0,1010.101010,1515.151515,2.0
Santa Ana la Real,19.0,3.0,7.0,475.0,631.578947,1473.684211,0.0
Beas,313.0,27.0,54.0,4341.0,621.976503,1243.953006,3.0
Punta Umbría,1091.0,76.0,153.0,15355.0,494.952784,996.418105,5.0
Isla Cristina,2677.0,108.0,204.0,21393.0,504.838031,953.582948,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Zalamea la Real,106.0,2.0,8.0,3054.0,65.487885,261.951539,0.0,0.250000
Encinasola,159.0,11.0,41.0,1310.0,839.694656,3129.770992,2.0,0.268293
Valverde del Camino,765.0,17.0,58.0,12750.0,133.333333,454.901961,4.0,0.293103
Nerva,232.0,9.0,29.0,5169.0,174.114916,561.036951,1.0,0.310345
Rosal de la Frontera,43.0,2.0,6.0,1697.0,117.855038,353.565115,0.0,0.333333
Palos de la Frontera,898.0,31.0,85.0,11742.0,264.009538,723.897121,10.0,0.364706
Cartaya,1583.0,32.0,86.0,20083.0,159.338744,428.222875,1.0,0.372093
Villarrasa,187.0,2.0,5.0,2211.0,90.456807,226.142017,0.0,0.400000
Galaroza,48.0,2.0,5.0,1382.0,144.717800,361.794501,0.0,0.400000
